#### LeetCode - SQL - #1407

Refer [1407. Top Travellers](https://leetcode.com/problems/top-travellers/description/)

Write a solution to report the distance traveled by each user.

Return the result table ordered by  `travelled_distance`  in  **descending order**, if two or more users traveled the same distance, order them by their  `name`  in  **ascending order**.

In [0]:
data_users = [[1, 'Alice'], [2, 'Bob'], [3, 'Alex'], [4, 'Donald'], [7, 'Lee'], [13, 'Jonathan'], [19, 'Elvis']]
columns_users = ['id', 'name']
schema_users = {'id':'Int64', 'name':'object'}

data_rides = [[1, 1, 120], [2, 2, 317], [3, 3, 222], [4, 7, 100], [5, 13, 312], [6, 19, 50], [7, 7, 120], [8, 19, 400], [9, 7, 230]]
columns_rides = ['id', 'user_id', 'distance']
schema_rides = {'id':'Int64', 'user_id':'Int64', 'distance':'Int64'}

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('LeetCode SQL').getOrCreate()

In [0]:
# Create Spark Dataframe using Pandas
import pandas

pandas_df_users = pandas.DataFrame(data=data_users, columns=columns_users).astype(schema_users)
pandas_df_rides = pandas.DataFrame(data=data_rides, columns=columns_rides).astype(schema_rides)

df_users = spark_context.createDataFrame(pandas_df_users)
df_users.printSchema()
df_users.show(5)

df_rides = spark_context.createDataFrame(pandas_df_rides)
df_rides.printSchema()
df_rides.show(5)

In [0]:
# Create Spark Dataframe using PySpark functions - 1
schema_users = 'id integer, name string'
schema_rides = 'id integer, user_id integer, distance long'

df_users = spark_context.createDataFrame(data=data_users, schema=schema_users)
df_users.printSchema()
df_users.show(5)

df_rides = spark_context.createDataFrame(data=data_rides, schema=schema_rides)
df_rides.printSchema()
df_rides.show(5)

In [0]:
from pyspark.sql import functions as F

df_users.join(
    df_rides,
    df_users.id == df_rides.user_id,
    "left"
).groupBy(df_users.id, df_users.name).agg(
    F.when(F.sum(df_rides.distance).isNotNull(), F.sum(df_rides.distance)).otherwise(0).alias('travelled_distance')
).select(
    F.col('name'),
    F.col('travelled_distance')
).orderBy(
    F.col('travelled_distance').desc(),
    F.col('name').asc()
).show()

+--------+------------------+
|    name|travelled_distance|
+--------+------------------+
|   Elvis|               450|
|     Lee|               450|
|     Bob|               317|
|Jonathan|               312|
|    Alex|               222|
|   Alice|               120|
|  Donald|                 0|
+--------+------------------+

